In [1]:
from collections import defaultdict
from typing import Iterable, Dict, List, Tuple

import numpy as np

from nupack import Complex as NupackComplex
from nupack import Model as NupackModel
from nupack import ComplexSet as NupackComplexSet
from nupack import Strand as NupackStrand
from nupack import SetSpec as NupackSetSpec
from nupack import complex_analysis as nupack_complex_analysis
from nupack import PairsMatrix as NupackPairsMatrix


import dsd.vienna_nupack as dv
import dsd.constraints as dc

# constants
DOMAIN_LENGTH = 15
DOMAIN_POOL: dc.DomainPool = dc.DomainPool('DOMAIN_POOL', DOMAIN_LENGTH)
TEMPERATURE = 37.0
NUPACK_MODEL = NupackModel(material='dna', celsius=TEMPERATURE)
NUMBER_OF_TRIALS = 1000

def nupack_base_pair_probabilities(strands: Iterable[dc.Strand], strands_to_assign: Iterable[dc.Strand], base_index1: int, base_index2: int) -> Tuple[np.ndarray, Dict[str, List[float]]]:
    """Given a design and a specific base pair, assigns random DNA sequences to
    the design and computes the equilibrium base-pairing probability for each
    DNA sequence assignment.
    
    :param strands: The strands that make up the design.
    :type strands: Iterable[dc.Strand]
    :param strands_to_assign: The strands to assign DNA to. Domains
        complementary to these strands are automatically reassigned DNA. As
        such, strands_to_assign is usually a subset of strands.
    :type strands: Iterabble[dc.Strand]
    :param base_index1: The index of one of the bases that form the base pair
    :type base_index1: int
    :param base_index2: The index of the other base that forms the base pair
    :type base_index2: int

    :return: An array of NUMBER_OF_TRIALS base pair probabilities and a
        dictionary which sorts the results by base (base located at
        base_index1)
    :rtype: Tuple[np.ndarray, Dict[str, List[float]]
    """
    for s in strands:
        for d in s.domains:
            d.pool = DOMAIN_POOL

    base_pair_probabilities = [0] * NUMBER_OF_TRIALS
    base_pair_probabilities_by_base: Dict[str, List[float]] = defaultdict(list)
    for i in range(NUMBER_OF_TRIALS):
        base_at_base_index1: str
        
        # Assign random DNA sequence
        for s in strands_to_assign:
            rand_dna_seq = dv.random_dna_seq(s.length())
            base_at_base_index1 = rand_dna_seq[base_index1]
            s.assign_dna(rand_dna_seq)
            
        # Initialize NUPACK NupackComplexSet
        nupack_strands = [NupackStrand(strand.sequence(), name=strand.name) for strand in strands]
        nupack_complex: NupackComplex = NupackComplex(nupack_strands)
        nupack_complex_set = NupackComplexSet(nupack_strands, complexes=(nupack_complex,))        
        
        # Call NUPACK complex_analysis        
        nupack_complex_result: np.ndarray = nupack_complex_analysis(nupack_complex_set, compute=['pairs'], model=NUPACK_MODEL)[nupack_complex].pairs.to_array()
            
        # Collect results
        p = nupack_complex_result[base_index1][base_index2]
        base_pair_probabilities[i] = p
        base_pair_probabilities_by_base[base_at_base_index1].append(p)
        
    return np.array(base_pair_probabilities), base_pair_probabilities_by_base

def summarize_bpps(bpps: np.ndarray, verbose: bool = False) -> None:
    if verbose:
        print('bpps:', bpps)
    print('mean', np.mean(bpps))
    print('standard deviation:', np.std(bpps))
    print()
    
def summarize_bpps_by_base(bpps_by_base: Dict[str, List[float]], verbose: bool = False) -> None:
    for base in 'ACGT':
        bpps = bpps_by_base[base]
        print('base:', base)
        summarize_bpps(np.array(bpps), verbose=verbose)

In [2]:
print('INTERIOR_TO_STRAND')
# INTERIOR_TO_STRAND
#                       a      b
#                     0  14 15  29
#                     |   |  |   |
#                    [-----##----->
#                     |||||  |||||
#                    <-----##-----]
#                     |   |  |   |
#                     59 45  44  30
#                       a*^   b*
#                         |
#                     base pair
top_strand: dc.Strand = dc.Strand(['a', 'b'], name='top strand')
bot_strand: dc.Strand = dc.Strand(['b*', 'a*'], name='bot strand')

interior_to_strand_bpps, interior_to_strand_bpps_by_base = nupack_base_pair_probabilities((top_strand, bot_strand), (top_strand,), 14, 45)
summarize_bpps(interior_to_strand_bpps)
summarize_bpps_by_base(interior_to_strand_bpps_by_base)

INTERIOR_TO_STRAND
mean 0.9961598146735028
standard deviation: 0.004097615093968879

base: A
mean 0.9930953115125866
standard deviation: 0.004797861934557825

base: C
mean 0.9989214263031253
standard deviation: 0.0008022386912883287

base: G
mean 0.9987576885463205
standard deviation: 0.0008673259258496431

base: T
mean 0.9938534680034865
standard deviation: 0.0037587604908127527



In [3]:
print('BLUNT_END')
#                       a
#                     0  14
#                     |   |
#                    [----->
#                     |||||
#                    <-----]
#                     |   |
#                     29  15
#
#                         ^
#                         |
#                     base pair

top_strand: dc.Strand = dc.Strand(['a'], name='top strand')
bot_strand: dc.Strand = dc.Strand(['a*'], name='bot strand')

blunt_end_bpps, blunt_end_bpps_by_base = nupack_base_pair_probabilities((top_strand, bot_strand), (top_strand,), 14, 15)
summarize_bpps(blunt_end_bpps)
summarize_bpps_by_base(blunt_end_bpps_by_base)

BLUNT_END
mean 0.47727186390081916
standard deviation: 0.1920031545877581

base: A
mean 0.285455398522373
standard deviation: 0.09602064421273485

base: C
mean 0.6361812967891943
standard deviation: 0.12207509241014516

base: G
mean 0.636431763510348
standard deviation: 0.13805214369629445

base: T
mean 0.35100897811251514
standard deviation: 0.028120727302013256



In [11]:
print('NICK_3P')
#                       a      b
#                     0  14 15  29
#                     |   |  |   |
#                    [----->[----->
#                     |||||  |||||
#                    <-----##-----]
#                     |   |  |   |
#                     59 45  44  30
#                       a*    b*
#                         ^
#                         |
#                     base pair
top_strand_left = dc.Strand(['a'], name='top strand left')
top_strand_right = dc.Strand(['b'], name='top strand right')
bot_strand = dc.Strand(['b*', 'a*'], name='bot strand')
strands = (top_strand_left, top_strand_right, bot_strand)
strands_to_assign = (bot_strand,)

nick_3p_bpps, nick_3p_bpps_by_base = nupack_base_pair_probabilities(strands, strands_to_assign, 14, 45)
summarize_bpps(nick_3p_bpps)
summarize_bpps_by_base(nick_3p_bpps_by_base)

NICK_3P
mean 0.8742121872885366
standard deviation: 0.10190680999062321

base: A
mean 0.8757657307483216
standard deviation: 0.10777071851850727

base: C
mean 0.8877222186725646
standard deviation: 0.09733371656693175

base: G
mean 0.8689141057764295
standard deviation: 0.10058028732155597

base: T
mean 0.8665374832241232
standard deviation: 0.1003975130259086



In [17]:
print('NICK_5P')
#                       a      b
#                     0  14 15  29
#                     |   |  |   |
#                    [-----##----->
#                     |||||  |||||
#                    <-----]<-----]
#                     |   |  |   |
#                     59 45  44  30
#                       a*    b*
#                         ^
#                         |
#                     base pair
top_strand: dc.Strand = dc.Strand(['a', 'b'], name='top strand')
bot_strand_right: dc.Strand = dc.Strand(['b*'], name='bot strand right')
bot_strand_left: dc.Strand = dc.Strand(['a*'], name='bot strand left')
strands = (top_strand, bot_strand_right, bot_strand_left)
strands_to_assign = (top_strand,)

# TODO(benlee12): Figure out why passing in (15,44) isn't matching above probabilities since they should be identical
nick_5p_bpps, nick_5p_bpps_by_base = nupack_base_pair_probabilities(strands, strands_to_assign, 14, 45)
summarize_bpps(nick_5p_bpps)
summarize_bpps_by_base(nick_5p_bpps_by_base)

NICK_5P
mean 0.8427234447418915
standard deviation: 0.1524696876467566

base: A
mean 0.6612009310997964
standard deviation: 0.14714646918739624

base: C
mean 0.9157392694438894
standard deviation: 0.08587892974345068

base: G
mean 0.9613272107142985
standard deviation: 0.04156451284815582

base: T
mean 0.8176977388861213
standard deviation: 0.10728554790266866



In [6]:
print('DANGLE_3P')
#                       a      b
#                     0  14 15  29
#                     |   |  |   |
#                    [-----##----->
#                     |||||
#                    <-----]
#                     |   |
#                     44 30
#                       a*
#                         ^
#                         |
#                     base pair
top_strand: dc.Strand = dc.Strand(['a', 'b'], name='top strand')
bot_strand: dc.Strand = dc.Strand(['a*'], name='bot strand')
strands = (top_strand, bot_strand)
strands_to_assign = (top_strand,)

dangle_3p_bpps, dangle_3p_bpps_by_base = nupack_base_pair_probabilities(strands, strands_to_assign, 14, 30)
summarize_bpps(dangle_3p_bpps)
summarize_bpps_by_base(dangle_3p_bpps_by_base)

DANGLE_3P
mean 0.6513151329670572
standard deviation: 0.20763460472969972

base: A
mean 0.4188259777123542
standard deviation: 0.1318610432319216

base: C
mean 0.744335583145423
standard deviation: 0.18787654111132535

base: G
mean 0.8318398214258101
standard deviation: 0.1216192390306667

base: T
mean 0.5917229541233645
standard deviation: 0.09804749423361542



In [7]:
print('DANGLE_5P')
#                       a
#                     0  14
#                     |   |
#                    [----->
#                     |||||
#                    <-----##-----]
#                     |   |  |   |
#                     44 30  29  15
#                       a*    b*
#                         ^
#                         |
#                     base pair
top_strand: dc.Strand = dc.Strand(['a'], name='top strand')
bot_strand: dc.Strand = dc.Strand(['b*', 'a*'], name='bot strand')
strands = (top_strand, bot_strand)
strands_to_assign = (bot_strand,)

dangle_5p_bpps, dangle_5p_bpps_by_base = nupack_base_pair_probabilities(strands, strands_to_assign, 14, 30)
summarize_bpps(dangle_5p_bpps)
summarize_bpps_by_base(dangle_5p_bpps_by_base)

DANGLE_5P
mean 0.674500957912793
standard deviation: 0.18894658410598036

base: A
mean 0.6632253311494264
standard deviation: 0.18948231937868687

base: C
mean 0.6667677247680522
standard deviation: 0.18893897979033974

base: G
mean 0.681097060503395
standard deviation: 0.1905587839864271

base: T
mean 0.68578659688443
standard deviation: 0.1856233755866417



In [8]:
print('DANGLE_5P_3P')
#                       a      b
#                     0  14 15  29
#                     |   |  |   |
#                    [-----##---->
#                     |||||
#                    <-----##----]
#                     |   |  |   |
#                     59 45  44  30
#                       a*    c
#                         ^
#                         |
#                     base pair
top_strand: dc.Strand = dc.Strand(['a', 'b'], name='top strand')
bot_strand: dc.Strand = dc.Strand(['c', 'a*'], name='bot strand')
strands = (top_strand, bot_strand)
strands_to_assign = (top_strand, bot_strand)

dangle_5p_3p_bpps, dangle_5p_3p_bpps_by_base = nupack_base_pair_probabilities(strands, strands_to_assign, 14, 45)
summarize_bpps(dangle_5p_3p_bpps)
summarize_bpps_by_base(dangle_5p_3p_bpps_by_base)

DANGLE_5P_3P
mean 0.8218638336744664
standard deviation: 0.17409232881782208

base: A
mean 0.8270410769069318
standard deviation: 0.17523137480731207

base: C
mean 0.8148222853016004
standard deviation: 0.18495021451105725

base: G
mean 0.8203705225927982
standard deviation: 0.16795135625748175

base: T
mean 0.8256158596349401
standard deviation: 0.167867956695125



In [9]:
print('OVERHANG_ON_THIS_STRAND_3P')
#                          ^
#                          |-29
#                          |   b
#                          |-15
#                          #
#                          #
#                       a  #    c
#                     0  14#  30  44
#                     |   |#  |   |
#                    [-----# [----->
#                     |||||   |||||
#                    <-----###-----]
#                     |   |   |   |
#                     74  60  59  45
#                       a*      c*
#                         ^
#                         |
#                     base pair
top_strand_left: dc.Strand = dc.Strand(['a', 'b'], name='top strand left')
top_strand_right: dc.Strand = dc.Strand(['c'], name='top strand right')
bot_strand: dc.Strand = dc.Strand(['c*', 'a*'], name='bot strand')
strands = (top_strand_left, top_strand_right, bot_strand)
strands_to_assign = (top_strand_left, bot_strand)

overhang_on_this_strand_3p_bpps, overhang_on_this_strand_3p_bpps_by_base = nupack_base_pair_probabilities(strands, strands_to_assign, 14, 60)
summarize_bpps(overhang_on_this_strand_3p_bpps)
summarize_bpps_by_base(overhang_on_this_strand_3p_bpps_by_base)

OVERHANG_ON_THIS_STRAND_3P
mean 0.8771440329103981
standard deviation: 0.12711078244529458

base: A
mean 0.8859052601973766
standard deviation: 0.11653610578119779

base: C
mean 0.8728632259897218
standard deviation: 0.1422259292040882

base: G
mean 0.8762068186864147
standard deviation: 0.12112580336363558

base: T
mean 0.8729298204767602
standard deviation: 0.1266073775259654



In [10]:
print('OVERHANG_ON_THIS_STRAND_5P')
#                     base pair
#                         |
#                         v
#                       a       b
#                     0   14  15  29
#                     |   |   |   |
#                    [-----###----->
#                     |||||   |||||
#                    <-----# <-----]
#                     |   |#  |   |
#                     74 60#  44  30
#                       a* #    b*
#                          #
#                          #
#                          |-59
#                          |    c
#                          |-45
#                          ]
top_strand: dc.Strand = dc.Strand(['a', 'b'], name='top strand')
bot_strand_right: dc.Strand = dc.Strand(['b*'], name='bot strand right')
bot_strand_left: dc.Strand = dc.Strand(['c', 'a*'], name='bot strand left')
strands = (top_strand, bot_strand_right, bot_strand_left)
strands_to_assign = (top_strand, bot_strand_left)

overhang_on_this_strand_5p_bpps, overhang_on_this_strand_5p_bpps_by_base = nupack_base_pair_probabilities(strands, strands_to_assign, 14, 60)
summarize_bpps(overhang_on_this_strand_5p_bpps)
summarize_bpps_by_base(overhang_on_this_strand_5p_bpps_by_base)

OVERHANG_ON_THIS_STRAND_5P
mean 0.856199969458763
standard deviation: 0.14940541602526836

base: A
mean 0.8617442853851801
standard deviation: 0.14595456839884396

base: C
mean 0.8487273473563436
standard deviation: 0.16918018566285606

base: G
mean 0.8516802527880221
standard deviation: 0.14993679600075402

base: T
mean 0.8633440705044365
standard deviation: 0.127601911803333

